# Student Depression Analysis

## Diagrammi per presentazione del 11.04.25

In [1]:
import pandas as pd
import plotly.express as px
df = pd.read_csv("data/student_depression_dataset.csv")


### Slide 3

In [2]:
# Author Davide Villa


# Mi aggrego i dati per gender e depressione
df_grouped = df.groupby(['Gender', 'Depression']).size().reset_index(name = 'count')

# Sostituisco il valore booleano con Yes e No
df_grouped['Depression'] = df_grouped['Depression'].map({1: 'Yes', 0: 'No'})

# Mi creo la series con il numero totale di persone per gender
total_per_gender = df_grouped.groupby('Gender')['count'].transform('sum')

# Calcolo la percentuale rispetto al totale del genere
df_grouped['percent'] = (df_grouped['count'] / total_per_gender * 100).round(1)

# Creo e mi salvo le etichette con la percentuale per i 4 gruppi
df_grouped['label'] = df_grouped.apply(lambda row: f"{row['percent']}%" , axis=1)

# Plot
fig = px.bar(
    data_frame = df_grouped,
    x = 'Gender',
    y = 'count',
    barmode = 'stack',
    color = 'Depression',
    template = 'presentation',
    title = '<b>Prevalence of depression by gender</b><br><sup>Percentage of depressed vs non-depressed</sup>',
    labels = {
        'count': 'Number of people',
        'Gender': 'Gender',
        'Depression': 'Depression'
    },
    color_discrete_map = {
        'Yes': 'red',
        'No': 'blue'
    },
    text = 'label'
)

# Configurazione font e posizione labels
fig.update_layout(
    margin = {
        't': 80,
        'r': 0,
        'b': 60,
        'l': 150
    },
    font = {
        'size': 25
    },
    legend = {
        'font': {'size': 30},
        'orientation': 'h'
    },
    xaxis = {
        'title_font' : {'size': 30},
        'tickfont': {'size': 25}
    },
    yaxis = {
        'title_font': {'size': 30},
        'title_standoff': 25,
        'tickfont': {'size': 25}
    }
)

fig.update_traces(
    textfont_size = 60,
    insidetextanchor = 'middle'
)

# Mostra figura
fig.show()


### Slide 4

In [3]:
# Author Giada Galdiolo

# Istogramma sovrapposto che mette in relazione work/study hours per Depression
fig = px.histogram(df, x="Work/Study Hours", color="Depression")
fig.update_layout(
    barmode="overlay",
    xaxis_range=[0, 12],
    
    # Titoli e font assi
    xaxis=dict(
        title="Ore di Studio/Lavoro",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title="Frequenza",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    
    # Titolo del grafico (opzionale)
    title=dict(
        text="Distribuzione Ore di Studio/Lavoro per Stato di Depressione",
        font=dict(size=20)
    )
)
fig.update_traces(opacity=0.75)


fig

### Slide 5

In [4]:
# Author Michea Colautti


# Filtra per considerare solo i livelli di Academic Pressure di interesse (es. 1, 2, 3, 4, 5)
df_filtered = df[df["Academic Pressure"].isin([1, 2, 3, 4, 5])]

# Crea il box plot: sull'asse X vengono mostrati i livelli di Academic Pressure e sull'asse Y il CGPA
fig = px.box(
    df_filtered,
    x="Academic Pressure",
    y="CGPA",
    title="Distribuzione del CGPA per livelli di Academic Pressure",
    labels={"Academic Pressure": "Academic Pressure", "CGPA": "CGPA"}
)

fig.update_layout(
    xaxis_range=[0.5, 5.5],
    yaxis_range=[4.9, 10.1],

    # Titoli e font assi
    xaxis=dict(
        title="Academic Pressure",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title="CGPA",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    )
)
fig.show()



### Slide 6


In [5]:
#Author everybod
import matplotlib.pyplot as plt

## Parte 1, città con valori fuori scala:

#Percentuale di studenti con depressione per città
city_depression_rate = df.groupby("City")["Depression"].mean().reset_index()
city_depression_rate["Depression"] = city_depression_rate["Depression"] * 100  # percentuale

fig = px.bar(city_depression_rate.sort_values("Depression", ascending=False),
              x="City", y="Depression",
              title="Depression Rate per City (%)",
              labels={"Depression": "Depression Rate (%)"})
fig.update_layout(xaxis_tickangle=-45)
fig.show()


## Parte 2, grafico solo con classificazioni.
# Piccola: < 3 milioni
# Media: 3–5 milioni
#Grande: > 5 milioni

# Crea un dizionario che mappa il nome della città alla classificazione.
city_classification = {
    "Delhi": "Grande",
    "Mumbai": "Grande",
    "Kolkata": "Grande",
    "Bangalore": "Grande",
    "Chennai": "Grande",
    "Hyderabad": "Grande",
    "Ahmedabad": "Grande",
    "Surat": "Grande",
    "Pune": "Grande",
    "Jaipur (Jalpur)": "Media",
    "Lucknow": "Media",
    "Indore": "Media",
    "Kanpur": "Media",
    "Nagpur": "Piccola",
    "Patna": "Piccola",
    "Thane": "Piccola",
    "Bnopar": "Piccola",
    "Visakhapatnam": "Piccola",
    "Vadodara": "Piccola",
    "Rajkot": "Piccola",
    "Faridabad": "Piccola",
    "Ludhiana": "Piccola",
    "Meerut": "Piccola",
    "Kalyan": "Piccola",
    "Vasai-Virar": "Piccola",
    "Nasnik": "Piccola",
    "Jalpur": "Piccola",
    "Varanas": "Piccola",
    "Ghaziabad": "Piccola",
    "Harsha": "Piccola"
}

# Crea la colonna 'CityClass'
df["CityClass"] = df["City"].map(city_classification)

# Raggruppa per (CityClass, Depressed) e conta gli elementi
counts_by_class = df.groupby(["CityClass", "Depression"]).size().unstack(fill_value=0)

# Somma per riga (totali per ogni CityClass)
row_sums = counts_by_class.sum(axis=1)

# Converti i conteggi in percentuali
percentages = (counts_by_class.T / row_sums).T * 100

# Per plot, facciamo un reset dell'indice in modo da avere colonne esplicite
percentages_reset = percentages.reset_index()

# Crea un grafico a barre "stacked" con Plotly Express
fig = px.histogram(
    percentages_reset,
    x="CityClass",
    y=[0, 1],
    barmode="group",
   labels={
        "CityClass": "Dimensione della Città",
        "value": "Percentuale (%)",
        "variable": "Stato di Depressione"
    },
    title="Percentuale di Persone Depresse vs Non Depresse per Dimensione di Città"
)


fig.update_layout(

    # Titoli e font assi
    xaxis=dict(
        title="Dimensione",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    ),
    yaxis=dict(
        title="Percentuale depressione",
        title_font=dict(size=18),
        tickfont=dict(size=14)
    )
)


fig.show()


